In [1]:
### Loading the data in

from biom import load_table
from biom.table import Table
import numpy as np
import pandas as pd
table = load_table('C:/Users/RensD/Documents/studie/table.biom')
print("made it")



made it


In [2]:
# Just looking around
print(table.head())
print(table.shape)
table.metadata()

# Constructed from biom file
#OTU ID	ERR1755873	ERR1755874	ERR1755875	ERR1755876	ERR1755877
3ba2c9b1b8d34808813fa03992ec6107bc153bfd	0.0	0.0	0.0	0.0	0.0
707a1cefe2a8fff1a72038b5f0b2149225e6716c	0.0	0.0	0.0	0.0	0.0
ea27b2e76524121fe0cccafd3c347646cf16d094	0.0	0.0	0.0	0.0	0.0
15e5107335f46cf2a5076fb793605ac82eb218f2	0.0	0.0	0.0	0.0	0.0
5fd09a74df8e2afcfa2a5c0f3efa215d8a18563a	0.0	0.0	0.0	0.0	0.0
(1256835, 1170)


(defaultdict(<function biom.table.Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda>()>,
             {'BioSample': 'SAMEA2732772',
              'Chlorophyll_Sensor': '0.052856',
              'Depth': '5',
              'Environment_Biome': 'marine biome (ENVO:00000447)',
              'Environment_Feature': 'surface water layer (ENVO:00002042)',
              'Environment_Material': 'particulate matter, including plankton (ENVO:xxxxxxxx)',
              'Environmental_Package': 'water',
              'Event_Date_Time_End': '2011-03-18T15:42',
              'Event_Date_Time_Start': '2011-03-18T15:26',
              'Experiment': 'ERX1823270',
              'InsertSize': '165',
              'Latitude_End': '-32.733',
              'Latitude_Start': '-32.7404',
              'Longitude_End': '-87.1005',
              'Longitude_Start': '-87.0992',
              'Marine_Region': '(search radius=5 degrees) Marine Ecoregion:Kermadec Island (MRGID:21914)',
              'Nitrat

In [48]:
table.metadata(axis="observation")

(defaultdict(<function biom.table.Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda>()>,
             {'chimera': 'N',
              'identity': 96.1,
              'taxonomy': ['Eukaryota', 'p', 'c', 'o', 'f', 'g', '*']}),
 defaultdict(<function biom.table.Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda>()>,
             {'chimera': 'N',
              'identity': 96.9,
              'taxonomy': ['Eukaryota',
               'p',
               'Dinophyceae',
               'Gymnodiniales',
               'Polykrikaceae',
               'Polykrikos',
               'Polykrikos schwartzii']}),
 defaultdict(<function biom.table.Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda>()>,
             {'chimera': 'N',
              'identity': 88.3,
              'taxonomy': ['Eukaryota', 'p', 'c', '*', 'f', 'g', '*']}),
 defaultdict(<function biom.table.Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda>()>,
             {'chimera': 'N',
            

In [9]:
### See how much samples have at least 5 abundance
total_abundances = table.sum(axis='observation') 
i = 0
minor_abundances = 0
major_abundances = 0
IDs = table.ids(axis="observation")
high_abundance_ids = list()

for abundance in total_abundances:
    if abundance <5:
        minor_abundances += 1
    if abundance >= 5:
        major_abundances += 1
        high_abundance_ids.append(IDs[i])
    i += 1
    
print("Abundances under 5 " + str(minor_abundances))
print("Abundances above 5 " + str(major_abundances))
print(len(high_abundance_ids))

### See how much samples are in at least 5% (58) of the samples
sample_abundances = table.nonzero_counts(axis="observation", binary=True)
i = 0
minor_abundances = 0
major_abundances = 0
high_count_ids = list()

for count in sample_abundances:
    if count <58:
        minor_abundances += 1
    if count >= 58:
        major_abundances += 1
        high_count_ids.append(IDs[i])
    i += 1
    
print("counts under 5% " + str(minor_abundances))
print("counts above 5% " + str(major_abundances))
print(len(high_count_ids))

Abbundances under 5 779498
Abbundances above 5 477337
477337
(477337, 1170)


In [47]:
total_abundances = table.sum(axis='observation') # List with the total abundance per OTU
sample_abundances = table.nonzero_counts(axis="observation", binary=True) # List with the amount of samples the OTU occurs in
IDs = table.ids(axis="observation") # List with all OTU identifiers
high_abundance_ids = list() # List to store IDs who qualify for all conditions
count_tresh = int(0.05 * table.shape[1]) # treshold to be consider abundant enough.

i = 0
for count in sample_abundances:
    if count >= count_tresh and total_abundances[i] >= 5:
        high_abundance_ids.append(IDs[i])
    i += 1
        
print("Amount of ID's that had a minimum abundance of 5 and are in at least 5% of samples")
print(len(high_abundance_ids))
    
new_table = table.filter(high_abundance_ids, inplace=False, axis='observation', invert = False)
print(new_table.shape)

filter_fn = lambda val, id_, md: md['Size_Fraction_Lower_Threshold'] == '0.8' # Filter on smallest sample size
new_table.filter(filter_fn, inplace=True, axis = "sample", invert = False)
print(new_table.shape)

Amount of ID's that had a minimum abundance of 5 and are in at least 5% of samples
25620
(25620, 1170)
(25620, 428)


### Abbundances summary
Total OTU's&emsp;&emsp;&emsp;1.256.835<br>
Abundance > 5 477377 &emsp; 38% left <br>
occurs in > 5% samples 25620 &emsp; 2% left <br>
both 25620 &emsp; 2% left.<br>
So the at least 5 abundancesrule is redundant here

In [41]:
filter_fn = lambda val, id_, md: md['Size_Fraction_Lower_Threshold'] == '0.8' # Filter on smallest sample size
print(table.shape)
new_table = table.filter(filter_fn, inplace=False, axis = "sample", invert = False)
print(new_table.shape)

(1256835, 1170)
(1256835, 428)


In [46]:
int(0.05 * table.shape[1])

58